In [8]:
import torch 
import torchvision 
import torch.nn as nn
import torchvision.transforms as transforms
import numpy as np

In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [10]:
#hyperparamter
input_size = 28*28
n_classes = 10 #output size
learning_rate = 0.001
batch_size = 64
num_epochs = 5

In [15]:
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()
])

In [17]:
# dataset -FashionMNIST

train_dataset = torchvision.datasets.CIFAR10(root="data/data/",
                                                  transform=transform,
                                                  train=True,
                                                  download=True)

Files already downloaded and verified


In [18]:
# dataset -FashionMNIST

test_dataset = torchvision.datasets.CIFAR10(root="data/data/",
                                                  transform=transform,
                                                  train=False)

In [13]:
# data loader 
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           shuffle=True,
                                           batch_size=batch_size)

In [14]:
# test data loader 
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size)

In [20]:
def conv3x3(in_channel,out_channel,stride=1):
    return nn.Conv2d(in_channels, 
                     out_channel, 
                     kernel_size=3, 
                     stride=stride, 
                     padding=1,
                     bias=False)

In [22]:
# model 
class ResNetBlock(nn.Module):
    def __init__(self, in_channel, out_channel, n_classes, stride=1, downsample=None):
        super(ResNetBlock,self).__init__()
        self.conv1 = conv3x3(in_channel, out_channel, stride)
        self.bn1 = nn.BatchNorm2d(out_channel)
        self.relu = nn.ReLU()
        self.conv2 = conv3x3(out_channel, out_channel)
        self.bn2 = nn.BatchNorm2d(out_channel)
        self.downsample = downsample
    def forward(self,x):
        residual = x
        out = self.conv(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out +=residual
        out = self.relu(out)
        return out

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, layers, n_classes):
        super(ResNet,self).__init__()
        

In [9]:
model = ConvNet(n_classes).to(device)

In [10]:
#loss and optimizer 
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [11]:
total_size = len(train_loader)
# Train the model
for epoch in range(num_epochs):
    for i,(images,labels) in enumerate(train_loader,0):
        images = images.to(device)
        labels = labels.to(device)
            #forward
        outputs = model(images)
        loss = criterion(outputs,labels)
        
        optimizer.zero_grad()
        
        #backpropagation
        loss.backward()
        optimizer.step()
        
        if (i+1)%500==0:
            print("Epoch {}/{} Step {}/{} : Loss {:.4f}".format(epoch+1,num_epochs,i+1, total_size,loss))

Epoch 1/5 Step 500/938 : Loss 0.3663
Epoch 2/5 Step 500/938 : Loss 0.2969
Epoch 3/5 Step 500/938 : Loss 0.2184
Epoch 4/5 Step 500/938 : Loss 0.3447
Epoch 5/5 Step 500/938 : Loss 0.1985


In [12]:
# test the model
with torch.no_grad():
    correct = 0.0
    total = 0.0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _,prediction = torch.max(outputs,1)
        total += labels.size(0)
        correct += (prediction==labels).sum()
    print("Accuracy : {:.2f}".format((correct*100.0)/total))

Accuracy : 90.00
